## Marginal Loss Cost Minimization Problem for Sustainability virtual Power Purchase

A Data Center company signs Power purchase agreements with a renewable project owner and development project. vPPA is the contract price that is Fixed wholesale Price /MWh between buyer and seller. The post project completion seller starts producing power to the grid and receives market price $/MWh. This market price is passed onto the buyer who has signed the vPPA. If the market price is less than agreed price, the Data Center pays the seller. If the market price is greater than the agreed price, the Data Center receives the difference. The buyer also receives renewable energy credits to meet sustainability goal for data center.


total system cost = utility cost for all zones for all data centers for year + regulatory cost + pLoss * demand

Minimize: 	System cost for entire system + (pLoss)* demand
pLoss = calculate loss for DC type of generation to satisfy dc  load
Subject to : 	supply demand constraint for each time stamp
		supply demand constraint for data center zones 



\begin{aligned}
min &\sum_{dc} \sum_{t=1}^{T} \sum_{z=1}^{Z} (utilitycost_{t,z,dc} + regulatorycost_{z,dc} + pLoss * demand_{t,z,dc}) \\
&\ pLoss_{t,z} = AgreePrice_{t,z} - CostMW_{t,z} \\
&\ s.t. \sum_{dc} (demand_{t,z,dc}) <= Load_{t,dc} \\
&\ utilitycost_{t,z,dc} >=0 \\
&\ regulatorycost_{z,dc} >=0 \\
&\ AgreePrice_{t,z} >=0 \\
&\ demand_{t,z,dc} >=0 \\
\end{aligned}



# Decision variables
1. $IBXCode_{dc}$, the data center IBX code;
2. $Loss_{DC}$, the loss for each data center;
# Sets
1. $DC$, the set of data centers; ${Dallas1, Dallas2}$ # ERCOT data centers$
2. $Demand$, the set of data center demand;


# Parameters
1. $utilitycost$, the utility cost for each data center;
2. $regulatorycost$, the regulatory cost for each data center;
3. $AgreePrice$, the vPPA price for each data center;
4. $CostMW$, the cost per MW for each data center;
5. $Load$, the load for each data center in 15 min interval;



In [2]:
# Imports
using CSV
using DataFrames
using Plots

In [3]:
dir = "Eastern_interconnect_2030/Power_System_Inputs_full_run/"

gen_data = CSV.read(joinpath(dir,"Generators_data.csv"), DataFrame);
fuels = CSV.read(joinpath(dir,"Fuels_data.csv"), DataFrame);
capacity = CSV.read(joinpath(dir,"Capacity_reserve_margin.csv"), DataFrame);
gen_variable = CSV.read(joinpath(dir,"Generators_variability.csv"), DataFrame);
energy_share = CSV.read(joinpath(dir,"Energy_share_requirement.csv"), DataFrame);
network_data = CSV.read(joinpath(dir,"Network.csv"), DataFrame);
load_data = CSV.read(joinpath(dir,"Load_data.csv"), DataFrame);

In [5]:
#This file outlines regional capacity reserve margin requirements. 
#Note that each region has a different reserve marginpercentage associated with it.

#Cap Reserve 3
Capacity_reserve_margin_PJM_EMAC_z10 = 0.156
Capacity_reserve_margin_PJM_Rest_z11 = 0.156

# 1 AND 2 ARE 0.0

# print CapRes1 ,2 3 for z10 and z11

for i in 1:3
    println("CapRes",i," z10 = ",capacity[i,2])
end

capacity



CapRes1 z10 = z1
CapRes2 z10 = z2
CapRes3 z10 = z3


Row,Column1,Network_zones,CapRes_1,CapRes_2,CapRes_3
,String15,String3,Float64,Float64,Float64
1,NENG_Rest,z1,0.193,0.0,0.0
2,NY_Z_A,z2,0.0,0.207,0.0
3,NY_Z_B,z3,0.0,0.207,0.0
4,NY_Z_C&E,z4,0.0,0.207,0.0
5,NY_Z_D,z5,0.0,0.207,0.0
6,NY_Z_F,z6,0.0,0.207,0.0
7,NY_Z_G-I,z7,0.0,0.207,0.0
8,NY_Z_J,z8,0.0,0.207,0.0
9,NY_Z_K,z9,0.0,0.207,0.0


In [32]:
#This file specifies regional policies for renewable portfolio standards and clean energy standards, 
#mandating a minimum level of energy generation from qualifying resources. Qualifying technologies are specifed further in the Generators_data file

# For ESR 3 and 4
Energy_share_requirement_PJM_EMAC_z10 = 0.372838
Energy_share_requirement_PJM_Rest_z11 = 0.188785

# For ESR 1, 2 5 and 6 is 0.0

energy_share

Row,Column1,Network_zones,ESR_1,ESR_2,ESR_3,ESR_4,ESR_5,ESR_6
,String15,String3,Float64,Float64,Float64,Float64,Float64,Float64
1,NENG_Rest,z1,0.344733,0.434508,0.0,0.0,0.0,0.0
2,NY_Z_A,z2,0.0,0.0,0.0,0.0,0.7,0.7
3,NY_Z_B,z3,0.0,0.0,0.0,0.0,0.7,0.7
4,NY_Z_C&E,z4,0.0,0.0,0.0,0.0,0.7,0.7
5,NY_Z_D,z5,0.0,0.0,0.0,0.0,0.7,0.7
6,NY_Z_F,z6,0.0,0.0,0.0,0.0,0.7,0.7
7,NY_Z_G-I,z7,0.0,0.0,0.0,0.0,0.7,0.7
8,NY_Z_J,z8,0.0,0.0,0.0,0.0,0.7,0.7
9,NY_Z_K,z9,0.0,0.0,0.0,0.0,0.7,0.7


In [48]:
#This file details fuel type, CO2 emissions intensity, and time-series data for fuel prices. Fuels are seperated by both technology type and region.

fuels[1:5,:]


Row,Time_Index,None,new_england_coal,new_england_naturalgas,new_england_uranium,middle_atlantic_naturalgas,middle_atlantic_uranium,middle_atlantic_coal,east_north_central_coal,east_north_central_naturalgas,east_north_central_uranium,new_england_naturalgas_ccs100,middle_atlantic_naturalgas_ccs100,east_north_central_naturalgas_ccs90,east_north_central_naturalgas_ccs100
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0,0.0,0.09552,0.05306,0.0,0.05306,0.0,0.09552,0.09552,0.05306,0.0,0.0,0.0,0.00531,0.0
2,1,0.0,0.0,3.81,0.79,3.37,0.79,2.2,2.1,3.73,0.79,3.81,3.37,3.73,3.73
3,2,0.0,0.0,3.81,0.79,3.37,0.79,2.2,2.1,3.73,0.79,3.81,3.37,3.73,3.73
4,3,0.0,0.0,3.81,0.79,3.37,0.79,2.2,2.1,3.73,0.79,3.81,3.37,3.73,3.73
5,4,0.0,0.0,3.81,0.79,3.37,0.79,2.2,2.1,3.73,0.79,3.81,3.37,3.73,3.73


In [36]:

#Network capacity parameters for z1 and z2 
Line_Max_Flow_MW_z10 = 0
Line_Min_Flow_MW_z1 = -150
Line_capacity_min_z1 = -800
Line_capacity_max_z2 = 800
Line_Loss_Percentage_z1 = 0.019
Line_Loss_Percentage_z2 = 0.019
Line_Reinforcement_Cost_per_MWyr_z1 =18641.0

network_data


Row,Network_zones,Network_Lines,z1,z2,z3,z4,z5,z6,z7,z8,z9,z10,z11,Line_Max_Flow_MW,Line_Min_Flow_MW,transmission_path_name,distance_mile,Line_Loss_Percentage,Line_Reinforcement_Cost_per_MWyr,Line_Max_Reinforcement_MW,DerateCapRes_1,DerateCapRes_2,DerateCapRes_3,CapRes_Excl_1,CapRes_Excl_2,CapRes_Excl_3
,String3?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,z1,1,1,0,0,0,-1,0,0,0,0,0,0,0.0,-150.0,NENG_Rest_to_NY_Z_D,190.217,0.019,18641.0,0.0,0.95,0.95,0.95,1,-1,0
2,z2,2,1,0,0,0,0,-1,0,0,0,0,0,800.0,-800.0,NENG_Rest_to_NY_Z_F,176.056,0.0176,17253.0,800.0,0.95,0.95,0.95,1,-1,0
3,z3,3,1,0,0,0,0,0,-1,0,0,0,0,600.0,-800.0,NENG_Rest_to_NY_Z_G-I,250.815,0.0251,24580.0,600.0,0.95,0.95,0.95,1,-1,0
4,z4,4,1,0,0,0,0,0,0,0,-1,0,0,760.0,-760.0,NENG_Rest_to_NY_Z_K,260.496,0.026,25528.0,760.0,0.95,0.95,0.95,1,-1,0
5,z5,5,0,1,-1,0,0,0,0,0,0,0,0,1930.0,-2270.0,NY_Z_A_to_NY_Z_B,54.242,0.0054,5316.0,1930.0,0.95,0.95,0.95,0,0,0
6,z6,6,0,1,0,0,0,0,0,0,0,0,-1,500.0,-500.0,NY_Z_A_to_PJM_Rest,254.214,0.0254,24913.0,500.0,0.95,0.95,0.95,0,1,-1
7,z7,7,0,0,1,-1,0,0,0,0,0,0,0,1300.0,-1300.0,NY_Z_B_to_NY_Z_C&E,94.5025,0.0095,9261.0,1300.0,0.95,0.95,0.95,0,0,0
8,z8,8,0,0,0,1,-1,0,0,0,0,0,0,1600.0,-2650.0,NY_Z_C&E_to_NY_Z_D,134.922,0.0135,13222.0,1600.0,0.95,0.95,0.95,0,0,0
9,z9,9,0,0,0,1,0,-1,0,0,0,0,0,3250.0,-1999.0,NY_Z_C&E_to_NY_Z_F,101.452,0.0101,9942.0,3250.0,0.95,0.95,0.95,0,0,0


In [37]:

#This file details both existing and new build resources for the model, including cost and performance data as well as storage and demand flexibility resources. Additionally, tags are included on specific resources so that policies can be applied when solving.

gen_data


Row,region,Resource,technology,cluster,R_ID,Zone,Num_VRE_Bins,CapRes_1,CapRes_2,THERM,VRE,MUST_RUN,STOR,FLEX,LDS,HYDRO,ESR_1,ESR_2,Min_Share,Max_Share,Existing_Cap_MWh,Existing_Cap_MW,Existing_Charge_Cap_MW,num_units,unmodified_existing_cap_mw,New_Build,Cap_Size,Min_Cap_MW,Max_Cap_MW,Max_Cap_MWh,Min_Cap_MWh,Max_Charge_Cap_MW,Min_Charge_Cap_MW,Min_Share_percent,Max_Share_percent,capex_mw,Inv_Cost_per_MWyr,Fixed_OM_Cost_per_MWyr,capex_mwh,Inv_Cost_per_MWhyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh,Var_OM_Cost_per_MWh_In,Inv_Cost_Charge_per_MWyr,Fixed_OM_Cost_Charge_per_MWyr,Start_Cost_per_MW,Start_Fuel_MMBTU_per_MW,Heat_Rate_MMBTU_per_MWh,heat_rate_mmbtu_mwh_iqr,heat_rate_mmbtu_mwh_std,voltage_level,Fuel,Min_Power,Self_Disch,Eff_Up,Eff_Down,Hydro_Energy_to_Power_Ratio,Ratio_power_to_energy,Min_Duration,Max_Duration,Max_Flexible_Demand_Delay,Max_Flexible_Demand_Advance,Flexible_Demand_Energy_Eff,Ramp_Up_Percentage,Ramp_Dn_Percentage,Up_Time,Down_Time,NACC_Eff,NACC_Peak_to_Base,Reg_Max,Rsv_Max,Reg_Cost,Rsv_Cost,spur_miles,spur_capex,offshore_spur_miles,offshore_spur_capex,tx_miles,tx_capex,interconnect_annuity,ids,HEAT,TRADER,NACC,Commit,Hydro_level,ESR_3,ESR_4,ESR_5,ESR_6,CapRes_3,MinCapTag_1,MinCapTag_2,MinCapTag_3,MinCapTag_4,MinCapTag_5,MinCapTag_6,MinCapTag_7,MinCapTag_8,MinCapTag_9,⋯
,String15,String,String,Int64,Int64,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String7,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,⋯
1,NENG_Rest,NENG_Rest_biomass_1,Biomass,1,1,1,0,0.93,0.0,0,0,1,0,0,0,0,1,1,0.0,0.0,0.0,43.3,0.0,5.0,161.8,-1,8.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,150850,0.0,0,0,12.55,0.0,0.0,0.0,0.0,0.0,29.2,3.126,3.358,trans,None,0.149,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,⋯
2,NENG_Rest,NENG_Rest_biomass_2,Biomass,2,2,1,0,0.93,0.0,0,0,1,0,0,0,0,1,1,0.0,0.0,0.0,150.0,0.0,55.0,560.505,-1,2.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,150850,0.0,0,0,4.68,0.0,0.0,0.0,0.0,0.0,10.88,2.765,1.401,trans,None,0.297,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,⋯
3,NENG_Rest,NENG_Rest_biomass_3,Biomass,3,3,1,0,0.93,0.0,0,0,1,0,0,0,0,1,1,0.0,0.0,0.0,153.9,0.0,22.0,574.992,-1,6.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,150850,0.0,0,0,7.43,0.0,0.0,0.0,0.0,0.0,17.3,3.508,2.487,trans,None,0.305,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,⋯
4,NENG_Rest,NENG_Rest_conventional_hydroelectric_1,Conventional Hydroelectric,1,4,1,0,0.8,0.0,0,0,0,0,0,0,1,0,1,0.0,0.0,0.0,1083.4,0.0,116.0,1083.44,-1,9.34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,53201,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,10.34,0.0,0.0,trans,None,0.258,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.083,0.083,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.5,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,⋯
5,NENG_Rest,NENG_Rest_conventional_steam_coal_1,Conventional Steam Coal,1,5,1,0,0.93,0.0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,534.6,0.0,4.0,534.6,0,133.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,86511,0.0,0,0,2.12,0.0,0.0,0.0,140.0,16.5,11.92,3.306,1.92,trans,new_england_coal,0.374,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.57,0.57,24,24,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0

In [49]:
#This file provides time-series data for load profiles in each model zone, along with associated weights, load shedding costs, and optional time domain reduction parameters.


load_data[1:5,:]


Row,Voll,Demand_Segment,Cost_of_Demand_Curtailment_per_MW,Max_Demand_Curtailment,$/MWh,Rep_Periods,Timesteps_per_Rep_Period,Sub_Weights,Time_Index,Load_MW_z1,Load_MW_z2,Load_MW_z3,Load_MW_z4,Load_MW_z5,Load_MW_z6,Load_MW_z7,Load_MW_z8,Load_MW_z9,Load_MW_z10,Load_MW_z11
,Int64?,Int64?,Float64?,Float64?,Int64?,Int64?,Int64?,Int64?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,9000,1,1.0,1.0,9000,1,8760,1,1,16921,1295,981,1943,222,1199,2146,6681,2115,17057,64043
2,missing,2,0.067,0.075,603,missing,missing,missing,2,16254,1247,944,1870,214,1154,2066,6419,2035,16311,61943
3,missing,missing,missing,missing,missing,missing,missing,missing,3,15781,1207,914,1811,207,1117,2001,6240,1981,15900,60760
4,missing,missing,missing,missing,missing,missing,missing,missing,4,15350,1163,880,1744,199,1075,1928,6038,1921,15776,60215
5,missing,missing,missing,missing,missing,missing,missing,missing,5,15175,1132,857,1698,194,1047,1878,5894,1877,15862,60241


$$  a^2  b^2_3 + \sigma $$